### Generating Quadrotor EoMs using Kane's Method and Sympy

![Drone image](imgs/drone.PNG)

Modeling a CrazyFlie 2.0 Drone 
Using Project guide from Temple University MEE 411/5411:
https://github.com/BAmercury/MobileRobotics_Project2/blob/1a99b41272352da760fc6c3ba6914aceb616297c/Project2-Phase1.pdf
Thanks to Dr. Phillip Dames

#### Given:
mass: m = 0.030 kg
Distance from CoM to axis of a motor: L = 0.046 m
Inertia in the body-fixed frame:
![Drone inertia matrix](imgs/drone_inertia.PNG)

#### Motor Model:

Each rotor has an angular speed $\omega$<sub>i</sub> and produces a vertical force F<sub>i</sub> according to this simple model:

F<sub>i</sub> = k<sub>F</sub> * $\omega$<sub>i</sub><sup>2</sup>

k<sub>F</sub> ~= 6.11 * 10<sup>-8</sup>N/rpm<sup>2</sup>

Rotors also produce a moment according to:

M<sub>i</sub> = k<sub>M</sub> * $\omega$<sub>i</sub><sup>2</sup>

with k<sub>M</sub> = 1.5 * 10<sup>-9</sup>Nm/rpm<sup>2</sup>

Through system ID, rotor speed can be related to commanded speed through this first order ODE:

$\dot{w}$ = k<sub>m</sub> * ($\omega$<sub>i</sub><sup>des</sup> - $\omega$<sub>i</sub>)

Motor gain k<sub>m</sub> was found to be 20s<sup>-1</sup> by matching performance of the simulation with the real system



In [1]:
from sympy.physics.mechanics import *
from sympy import symbols, trigsimp
from numpy import deg2rad, rad2deg, array, zeros, linspace, pi
from scipy.integrate import odeint
from sympy.functions.elementary.trigonometric import sin, cos
from sympy import Matrix
from pydy.codegen.ode_function_generators import generate_ode_function

from sympy.physics.vector import init_vprinting
import matplotlib.pyplot as plt


In [180]:
# Define state vector

# ZXY Euler angles: Roll Pitch Yaw Euler angles in inertial frame
theta, phi, psi = dynamicsymbols('theta phi psi') # drone orientation expressed in inertial frame
thetadot, phidot, psidot = dynamicsymbols('theta phi psi', 1) # drone angular velocity expressed in inertial frame

# Body rates 
p, q, r = dynamicsymbols('p q r') # drone angular velocity in the body frame
pdot, qdot, rdot = dynamicsymbols('p q r', 1) 

# X Y Z translation in inertial frame
x, y, z = dynamicsymbols('x y z')
xd, yd, zd = dynamicsymbols('x y z', 1)
# Generalized speeds
# Define the translation speeds in the inertial frame
# Define the rotational speeds in the body frame to simplify the process, otherwise the equations come out pretty ugly
# u1 = xd
# u2 = yd
# u3 = zd
# u4 = p
# u5 = q
# u6 = r
u1, u2, u3, u4, u5, u6 = dynamicsymbols('u1 u2 u3 u4 u5 u6')
u1d, u2d, u3d, u4d, u5d, u6d = dynamicsymbols('u1 u2 u3 u4 u5 u6', 1)


# Define Reference Frames
Nrf = ReferenceFrame('Nrf') # Inertial Reference Frame
# Body Fixed Reference Frame
# Z-X-Y Euler rotation, will write the DCM directly for sympy
cpsi = cos(psi)
spsi = sin(psi)
ctheta = cos(theta)
stheta = sin(theta)
cphi = cos(phi)
sphi = sin(phi)

c11 = cpsi*ctheta - sphi*spsi*stheta
c12 = -cphi*spsi
c13 = cpsi*stheta + ctheta*spsi*sphi
c21 = ctheta*spsi + cpsi*sphi*stheta
c22 = cphi*cpsi
c23 = spsi*stheta - cpsi*ctheta*sphi
c31 = -cphi*stheta
c32 = sphi
c33 = cphi*ctheta

BrfToNrf = Matrix([[c11, c12, c13],
                   [c21, c22, c23],
                   [c31, c32, c33]])

# Euler Kinematical Equations matrix
inertial2bodyrates = Matrix([[ctheta, 0, -cphi*stheta],
                             [0,      1,  sphi],
                             [stheta, 0,  cphi*ctheta]])

# Body frame is attached to CoM of drone
# b1 coincides with forward direction
# b3 is perpendicular ot plane of rotors pointing vertically up during hover
# body frame is parallel to principal axes
Brf = Nrf.orientnew('Brf', 'DCM', BrfToNrf)

# Set angular velocity of these frames
# Set angular velocity of Brf wrt to Nrf, express the angular velocity in the body frame i.e b_w_b/n
Brf.set_ang_vel(Nrf, u4*Brf.x + u5*Brf.y + u6*Brf.z)

# Define Locations
O = Point('O') # Inertial origin
O.set_vel(Nrf, 0)

# CoM of Drone
d_CG = Point('d_CG')
# Set the CG position and velocity
# Express Position and Velocity in inertial frame
d_CG.set_pos(O, x*Nrf.x + y*Nrf.y + z*Nrf.z)
# Define using generalized speeds
d_CG.set_vel(Nrf, u1*Nrf.x + u2*Nrf.y + u3*Nrf.z)
# Location of rotor motors
l = symbols('l') # length between drone cg and motor location
M1p = d_CG.locatenew('M1p', l*Brf.x)
M2p = d_CG.locatenew('M2p', l*Brf.y)
M3p = d_CG.locatenew('M3p', -l*Brf.x)
M4p = d_CG.locatenew('M4p', -l*Brf.y)

M1p.v2pt_theory(d_CG, Nrf, Brf)
M2p.v2pt_theory(d_CG, Nrf, Brf)
M3p.v2pt_theory(d_CG, Nrf, Brf)
M4p.v2pt_theory(d_CG, Nrf, Brf)

# Gather everything into lists
# X Y Z translation
coordinates = [x, y, z, phi, theta, psi]
speeds = [u1, u2, u3, u4, u5, u6]

# Kinematic Differential Equations

# Kinematic relationship between inertial rates and body rates
# u4, u5, u6 are p, q r in the body frame
# The coordinates are in the inertial frame, so we need to convert the derivatives of the coordinates (theta, psi, phi) to the body frame
pqr_vec = inertial2bodyrates * Matrix([phidot, thetadot, psidot])

kde = [u1 - xd, u2 - yd, u4 - zd, u4 - pqr_vec[0], u5 - pqr_vec[1], u6 - pqr_vec[2]]


# initial kane's object
kane = KanesMethod(Nrf, q_ind=coordinates, u_ind=speeds, kd_eqs =kde)

# inertia dynamics
# inertia(N, 1, 2, 3, 4, 5, 6).to_matrix(N)
# Matrix([
#[1, 4, 6],
#[4, 2, 5],
#[6, 5, 3]])
I_x, I_y, I_z, m_drone = symbols('I_X I_Y I_Z m_drone') # Inertia of drone and mass 
#itd = inertia(Brf, 1.43e-5, 1.43e-5, 2.89e-5, 0, 0, 0)
itd = inertia(Brf, I_x, I_y, I_z)

i1 = (itd, d_CG)
# Define rigid body objects
drone_body = RigidBody('drone_body', d_CG, Brf, m_drone, i1)

# Define forces and point of application
# Gravity
g = symbols('g') # gravity
# Thrust and Torque from each motor 
F1, F2, F3, F4, M1, M2, M3, M4 = dynamicsymbols('F1 F2 F3 F4 M1 M2 M3 M4') # Make these dynamic as they can be time varying inputs
f_g = (d_CG, m_drone*g*Nrf.z)
f_M1 = (M1p, -F1*Brf.z)
f_M2 = (M2p, -F2*Brf.z)
f_M3 = (M3p, -F3*Brf.z)
f_M4 = (M4p, -F4*Brf.z)

t_M1 = (Brf, -M1*Brf.z) # CCW
t_M2 = (Brf, M2*Brf.z) # CW
t_M3 = (Brf, -M3*Brf.z) # CCW
t_M4 = (Brf, M4*Brf.z) # CW

loads = [f_g, f_M1, f_M2, f_M3, f_M4, t_M1, t_M2, t_M3, t_M4]
bodies = [drone_body]





In [26]:
d_CG.pos_from(O).dt(Nrf)

Derivative(x(t), t)*Nrf.x + Derivative(y(t), t)*Nrf.y + Derivative(z(t), t)*Nrf.z

In [181]:
kde

[u1(t) - Derivative(x(t), t),
 u2(t) - Derivative(y(t), t),
 u4(t) - Derivative(z(t), t),
 u4(t) + sin(theta(t))*cos(phi(t))*Derivative(psi(t), t) - cos(theta(t))*Derivative(phi(t), t),
 u5(t) - sin(phi(t))*Derivative(psi(t), t) - Derivative(theta(t), t),
 u6(t) - sin(theta(t))*Derivative(phi(t), t) - cos(phi(t))*cos(theta(t))*Derivative(psi(t), t)]

In [28]:
pqr_vec = inertial2bodyrates * Matrix([phidot, thetadot, psidot])
pqr_vec[1]


sin(phi(t))*Derivative(psi(t), t) + Derivative(theta(t), t)

In [29]:
# Should return Euler's Kinematical Equations
# [p, q, r] = H * [phi_d, theta_d, psi_d]
Nrf.ang_vel_in(Brf)

- u4(t)*Brf.x - u5(t)*Brf.y - u6(t)*Brf.z

In [30]:
Brf.ang_vel_in(Nrf)

u4(t)*Brf.x + u5(t)*Brf.y + u6(t)*Brf.z

In [31]:
# Returns Brf to Nrf DCM
Brf.dcm(Nrf)

Matrix([
[-sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(psi(t))*cos(theta(t)),  sin(phi(t))*sin(theta(t))*cos(psi(t)) + sin(psi(t))*cos(theta(t)), -sin(theta(t))*cos(phi(t))],
[                                          -sin(psi(t))*cos(phi(t)),                                            cos(phi(t))*cos(psi(t)),                sin(phi(t))],
[ sin(phi(t))*sin(psi(t))*cos(theta(t)) + sin(theta(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t)),  cos(phi(t))*cos(theta(t))]])

In [19]:
BrfToNrf

Matrix([
[-sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(psi(t))*cos(theta(t)), -sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t))*cos(theta(t)) + sin(theta(t))*cos(psi(t))],
[ sin(phi(t))*sin(theta(t))*cos(psi(t)) + sin(psi(t))*cos(theta(t)),  cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t))],
[                                        -sin(theta(t))*cos(phi(t)),              sin(phi(t)),                                          cos(phi(t))*cos(theta(t))]])

In [182]:
fr, frstar = kane.kanes_equations(bodies, loads)
trigsimp(fr + frstar)

Matrix([
[-m_drone*Derivative(u1(t), t) - F1(t)*sin(phi(t))*sin(psi(t))*cos(theta(t)) - F1(t)*sin(theta(t))*cos(psi(t)) - F2(t)*sin(phi(t))*sin(psi(t))*cos(theta(t)) - F2(t)*sin(theta(t))*cos(psi(t)) - F3(t)*sin(phi(t))*sin(psi(t))*cos(theta(t)) - F3(t)*sin(theta(t))*cos(psi(t)) - F4(t)*sin(phi(t))*sin(psi(t))*cos(theta(t)) - F4(t)*sin(theta(t))*cos(psi(t))],
[            -m_drone*Derivative(u2(t), t) - (-sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t)))*F1(t) - (-sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t)))*F2(t) - (-sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t)))*F3(t) - (-sin(phi(t))*cos(psi(t))*cos(theta(t)) + sin(psi(t))*sin(theta(t)))*F4(t)],
[                                                                                                                                                                             g*m_drone - m_drone*Derivative(u3(t), t) - F1(t)*cos(phi(t))*cos(theta(t)) - F2(t)*cos(phi(t))*cos(theta(t)) - 

In [199]:
# Linearize about hover
# x, y, z = [x0, y0, z0]
# linearize(op_point=None, A_and_B=False, simplify=False)
x0, y0, z0, psi0 = symbols('x0 y0 z0 psi0')
op_point = {
    x: x0,
    y: y0,
    z: z0,
    F1: (m_drone*g)/4,
    F2: (m_drone*g)/4,
    F3: (m_drone*g)/4,
    F4: (m_drone*g)/4,
    phidot: 0,
    thetadot: 0,
    psidot: 0,
    u4: 0,
    u5: 0,
    u6: 0,
    theta: 0,
    phi: 0,
    psi: psi0,
    u1: 0,
    u2: 0,
    u3: 0,
}
# oordinates = [x, y, z, phi, theta, psi]
# speeds = [u1, u2, u3, u4, u5, u6]
linearizer  = kane.to_linearizer()

linearizer.r = Matrix([F1, F2, F3, F4, M1, M2, M3, M4]) # can use to reorganize inputs
A, B = linearizer.linearize(A_and_B=True, op_point=[op_point, simplify = False)


In [200]:
trigsimp(A*Matrix([linearizer.q, linearizer.u]))

Matrix([
[                                     u1(t)],
[                                     u2(t)],
[                                     u4(t)],
[                                     u4(t)],
[                                     u5(t)],
[                                     u6(t)],
[-g*(phi(t)*sin(psi0) + theta(t)*cos(psi0))],
[ g*(phi(t)*cos(psi0) - theta(t)*sin(psi0))],
[                                         0],
[                                         0],
[                                         0],
[                                         0]])

In [201]:
trigsimp(B*linearizer.r)

Matrix([
[                                                             0],
[                                                             0],
[                                                             0],
[                                                             0],
[                                                             0],
[                                                             0],
[                                                             0],
[                                                             0],
[-F1(t)/m_drone - F2(t)/m_drone - F3(t)/m_drone - F4(t)/m_drone],
[                                    -l*F2(t)/I_X + l*F4(t)/I_X],
[                                     l*F1(t)/I_Y - l*F3(t)/I_Y],
[                -M1(t)/I_Z + M2(t)/I_Z - M3(t)/I_Z + M4(t)/I_Z]])

In [202]:
trigsimp(A*Matrix([linearizer.q, linearizer.u]) + B*linearizer.r )


Matrix([
[                                                         u1(t)],
[                                                         u2(t)],
[                                                         u4(t)],
[                                                         u4(t)],
[                                                         u5(t)],
[                                                         u6(t)],
[                    -g*(phi(t)*sin(psi0) + theta(t)*cos(psi0))],
[                     g*(phi(t)*cos(psi0) - theta(t)*sin(psi0))],
[-F1(t)/m_drone - F2(t)/m_drone - F3(t)/m_drone - F4(t)/m_drone],
[                                    -l*F2(t)/I_X + l*F4(t)/I_X],
[                                     l*F1(t)/I_Y - l*F3(t)/I_Y],
[                -M1(t)/I_Z + M2(t)/I_Z - M3(t)/I_Z + M4(t)/I_Z]])